# Groupe non officiel 1
# Livrable 2 - Classification binaire Création du model

|Auteur|Centre|
|---|---|
|ACQUART Quentin|Aix-en-Provence|
|DIMEGLIO Nicolas|Aix-en-Provence|
|ROMANO Sébastien|Aix-en-Provence|
|SIXDENIER Alexandre|Aix-en-Provence|
|VESSERON Alexandre|Aix-en-Provence|

## Rappel du sujet
L'entreprise voulant automatiser la sélection de photos pour l'annotations, le but est de fournir une méthode de classification binaire à fin de filtrer les images qui ne sont pas des photos du dataset de départ. Pour ce faire nous allons nous appuyer sur l'architecture des réseaux de neurones, ainsi que l'analyse des résultats obtenus.
Toutes les parties doivent être détaillées dans le notebook :
    - les paramètre du réseau,
    - la fonction de perte ainsi que l'algorithme d'optimisation utilisé pour l’entraînement.

## Analyse à effectuer

Le but de cette étape va être le suivant :
    Traiter l'ensemble des photographies via un bruitage puis un débruitage à fin d'améliorer leur qualité.
    Le tout à l'aide d'un auto-encodeur à convolution.


## Import des différentes bibliothèques

In [16]:
import pathlib
import zipfile
import os
import tensorflow as tf
from PIL import Image as Image
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras import regularizers
from sklearn.model_selection import train_test_split

## Téléchargement des données et dezippage
Pour cette partie se referrer au notebook [Préparation des données](Préparation_des_données.ipynb#section_1)
## Récupération des données
Une fois les datasets mis en place, sélectionner le dataset à utiliser dans la variable `data_dir`

In [17]:
clean_dataset = "../DatasetL2/Project_Dataset_Clean"

data_dir = clean_dataset

data_dir = pathlib.Path(data_dir)

X_train, _, y_train, _ = train_test_split(data_dir,data_dir, test_size=0.33)

TypeError: Singleton array array(WindowsPath('../DatasetL2/Project_Dataset_Clean'), dtype=object) cannot be considered a valid collection.

In [3]:
image_h = 180
image_w = 180
batch_s = 32

In [ ]:
# Lisser le poids des images

encoding_dim = 32

input_img = keras.Input(shape=(784,))
# Add a Dense layer with a L1 activity regularizer
encoded = layers.Dense(encoding_dim, activation='relu',
                       activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)